In [ ]:
import csv
import os
import glob
from pathlib import Path
current_file_dir = os.path.dirname(os.path.realpath("__file__"))
for i in glob.glob(os.path.join(current_file_dir,"logs","*.txt")):
    print(i)
    baselogfile = os.path.basename(i)
    print(baselogfile)
    basefilename = os.path.splitext(baselogfile)[0]
    output_csv_losslog_name = basefilename+"_loss.csv" 
    output_csv_mAPlog_name = basefilename+"_mAP.csv"
    print(output_csv_mAPlog_name)
    output_csv_log_dir = os.path.join(current_file_dir,"csvlog")
    if not os.path.exists(output_csv_log_dir):
        os.makedirs(output_csv_log_dir)

    with open(i, 'rb') as file_in:
        #Read logs and filter out the terms
        line = list(filter(lambda line: (b'loss =' in line and b'step =' in line), file_in))
        loss_step_list=[line[i].decode("utf-8").split(":")[-1] for i in range(0,len(line),2)] 
        #print(loss_step_list)
        mAP_Dict = {}
        step_loss={}
        for i in loss_step_list:
            if 'global_step =' in i:
                #Extract mAP values and steps
                #print("Global step section")
                mAP = i.split(",")
                #print(i)
                mAP_value = mAP[0].split(" = ")[1]
                mAP_step = mAP[-3].split(" = ")[1]
                mAP_Dict[str(mAP_step)]=mAP_value
            else:
                loss_step = i.split("(")[0]
                #Strip the string of loss and steps
                loss,step = loss_step.split(",")[0].strip(),loss_step.split(",")[1].strip()
                loss_stripped = loss.split(" = ")[-1]
                step_stripped = step.split(" = ")[-1]
                step_loss[str(step_stripped)]=loss_stripped
        print(mAP_Dict)
        #print(" ")
        print(step_loss)
        #Writes log into csv log directory
        loss_log_file = os.path.join(output_csv_log_dir,output_csv_losslog_name)
        with open(loss_log_file, "w+") as f1:  # Just use 'w' mode in 3.x
            w = csv.writer(f1)
            w.writerow(["steps","losses"])
            w.writerows(step_loss.items())
        mAP_log_file = os.path.join(output_csv_log_dir,o
                                    utput_csv_mAPlog_name)
        with open(mAP_log_file,"w+") as f2:
            w = csv.writer(f2)
            w.writerow(["steps","Average_Precision"])
            w.writerows(mAP_Dict.items())

In [62]:
#Updated to suit changes in model_main.py with added time,start,end
import csv
import os
import glob
from pathlib import Path
current_file_dir = os.path.dirname(os.path.realpath("__file__"))
for i in glob.glob(os.path.join(current_file_dir,"logs","*.log")):
    print(i)
    baselogfile = os.path.basename(i)
    print(baselogfile)
    basefilename = os.path.splitext(baselogfile)[0]
    output_csv_losslog_name = basefilename+"_loss.csv" 
    output_csv_mAPlog_name = basefilename+"_mAP.csv"
    output_csv_durationlog_name = basefilename+"_duration.csv"
    output_csv_log_dir = os.path.join(current_file_dir,"csvlog")
    #Make directory if it does not exists
    if not os.path.exists(output_csv_log_dir):
        os.makedirs(output_csv_log_dir)

    with open(i, 'r') as file_in:
        #Read logs and filter out the terms
        list_format = list(file_in)
        #print(list_format)
        loss_step_list = [i for i in list_format if (' loss = ' in i and ' step = ' in i) or 'Saving dict for global step' in i or ' Loss for final step:' in i]
        print("Loss step list: {}".format(loss_step_list)) 
        time_start_end_info = [i for i in list_format if ' Started:' in i or ' Ended:' in i or ' Duration:' in i]
        #print(time_start_end_info)
        mAP_list = [i for i in list_format if 'Saving dict for global step ' in i ]
        #print(mAP_list)

        mAP_dict = {}
        step_loss={}
        time_dict = {}
        for i in time_start_end_info:
            if "Started:" in i:
                start_time = i.split(":",1)[1].strip()
                print(start_time)
                time_dict["Started"]= str(start_time)
            if "Ended:" in i:
                end_time = i.split(":",1)[1].strip()
                print(end_time)
                time_dict["Ended"]= str(end_time)
            if "Duration:" in i:
                duration = i.split(":",1)[1].strip().split(" ")[0]
                print(duration)
                time_dict["Duration"] = str(duration)

        for i in mAP_list:
            if 'global_step =' in i:
                #Extract mAP values and steps for each entry
                #print("Global step section")
                mAP = i.split(",")
                #print(mAP)
                mAP_value = mAP[0].split(" = ")[1]
                print("mAP_value : {}".format(mAP_value))
                mAP_step = mAP[-3].split(" = ")[1]
                print("mAP_step : {}".format(mAP_step))
                mAP_dict[str(mAP_step)]=mAP_value
                
        for i in loss_step_list:
            
            if "INFO - loss =" in i and "step =" in i:
                loss_step_content = i.split("(")[0]
                print("Loss_content: {}".format(loss_step_content))
                loss,step = loss_step_content.split(",")[0].split("=")[1].strip(),loss_step_content.split(",")[1].split("=")[1].strip()
                print("Loss: {}".format(loss))
                print("Step: {}".format(step))
                step_loss[step]= loss
            if "Loss for final step:" in i:
                last_step_value= str(max(mAP_dict.keys()))
                final_loss= i.split(":")[1].strip()
                #print(final_loss)
                print("mAP_list: {}".format(mAP_list))
                step_loss[last_step_value] = final_loss
        print(mAP_dict)
        print("Step loss: {}".format(step_loss))
        print(time_dict)
 
        #Writes log into csv log directory
        
        loss_log_file = os.path.join(output_csv_log_dir,output_csv_losslog_name)
        with open(loss_log_file, "w+") as f1:  # Just use 'w' mode in 3.x
            w = csv.writer(f1)
            w.writerow(["steps","losses"])
            w.writerows(step_loss.items())
        mAP_log_file = os.path.join(output_csv_log_dir,output_csv_mAPlog_name)
        with open(mAP_log_file,"w+") as f2:
            w = csv.writer(f2)
            w.writerow(["steps","Average_Precision"])
            w.writerows(mAP_dict.items())
        durationlog_file = os.path.join(output_csv_log_dir,output_csv_durationlog_name)
        with open(durationlog_file,"w+") as f3:
            w = csv.writer(f3)
            w.writerow(["Start","End","Duration(seconds)"])
            print(time_dict.values())
            w.writerow([time_dict["Started"],time_dict["Ended"],time_dict["Duration"]])

/home/qzq/Documents/Dockers/tf_objectdetection/Vedai/logs/output_rfcn_resnet_variant_50k.log
output_rfcn_resnet_variant_50k.log
Loss step list: ['INFO - loss = 3.680578, step = 0\n', 'INFO - loss = 2.668031, step = 100 (75.389 sec)\n', 'INFO - Saving dict for global step 200: DetectionBoxes_Precision/mAP = 0.0, DetectionBoxes_Precision/mAP (large) = 0.0, DetectionBoxes_Precision/mAP (medium) = 0.0, DetectionBoxes_Precision/mAP (small) = 0.0, DetectionBoxes_Precision/mAP@.50IOU = 0.0, DetectionBoxes_Precision/mAP@.75IOU = 0.0, DetectionBoxes_Recall/AR@1 = 0.0, DetectionBoxes_Recall/AR@10 = 0.0, DetectionBoxes_Recall/AR@100 = 0.0, DetectionBoxes_Recall/AR@100 (large) = 0.0, DetectionBoxes_Recall/AR@100 (medium) = 0.0, DetectionBoxes_Recall/AR@100 (small) = 0.0, Loss/BoxClassifierLoss/classification_loss = 1.5329058, Loss/BoxClassifierLoss/localization_loss = 0.0005072113, Loss/RPNLoss/localization_loss = 0.20861152, Loss/RPNLoss/objectness_loss = 0.70668024, Loss/total_loss = 2.4487047, 